In [1]:
import requests
import pandas as pd
import numpy as np
import json

In [2]:
counter = 0

In [3]:
#Query all data from a year
#Add counter for the number of extractions
data=[]
for i in range(1,30):
    counter+=1
    url = "https://api-football-v1.p.rapidapi.com/v3/players"

    querystring = {"league":"39","season":"2015","page":str(i)}

    headers = {
        "X-RapidAPI-Host": "api-football-v1.p.rapidapi.com",
        "X-RapidAPI-Key": "d356c39fe7mshf3ec31c0312f244p17df85jsn2bc9fd933f3f"
   }

    response = requests.request("GET", url, headers=headers, params=querystring)
    
    if response.text == "'{'message': 'You have exceeded the rate limit per minute for your plan, BASIC, by the API provider'}'":
        print('Restart next loop at:', i)
        break
    else: 
        data.append(response.text)
else: 
    print('Start next loop at 30')
data
#Good but have to pop last pages which were over my quota per minute

Start next loop at 30


['{"get":"players","parameters":{"league":"39","page":"1","season":"2015"},"errors":[],"results":20,"paging":{"current":1,"total":38},"response":[{"player":{"id":18015,"name":"Elliott Ward","firstname":"Elliott","lastname":"Ward","age":36,"birth":{"date":"1985-01-19","place":"London","country":"England"},"nationality":"England","height":"185 cm","weight":"83 kg","injured":false,"photo":"https:\\/\\/media.api-sports.io\\/football\\/players\\/18015.png"},"statistics":[{"team":{"id":35,"name":"Bournemouth","logo":"https:\\/\\/media.api-sports.io\\/football\\/teams\\/35.png"},"league":{"id":39,"name":"Premier League","country":"England","logo":"https:\\/\\/media.api-sports.io\\/football\\/leagues\\/39.png","flag":"https:\\/\\/media.api-sports.io\\/flags\\/gb.svg","season":2015},"games":{"appearences":0,"lineups":0,"minutes":0,"number":null,"position":"Defender","rating":null,"captain":false},"substitutes":{"in":0,"out":0,"bench":0},"shots":{"total":null,"on":null},"goals":{"total":0,"conce

In [4]:
counter

29

In [5]:
#complete year array
for i in range(30,39):
    counter+=1
    url = "https://api-football-v1.p.rapidapi.com/v3/players"
    
    #YEAR TO CHANGE IN QUERY
    querystring = {"league":"39","season":"2015","page":str(i)}

    headers = {
        "X-RapidAPI-Host": "api-football-v1.p.rapidapi.com",
        "X-RapidAPI-Key": "d356c39fe7mshf3ec31c0312f244p17df85jsn2bc9fd933f3f"
    }

    response = requests.request("GET", url, headers=headers, params=querystring)
    
    data.append(response.text)
    
data

['{"get":"players","parameters":{"league":"39","page":"1","season":"2015"},"errors":[],"results":20,"paging":{"current":1,"total":38},"response":[{"player":{"id":18015,"name":"Elliott Ward","firstname":"Elliott","lastname":"Ward","age":36,"birth":{"date":"1985-01-19","place":"London","country":"England"},"nationality":"England","height":"185 cm","weight":"83 kg","injured":false,"photo":"https:\\/\\/media.api-sports.io\\/football\\/players\\/18015.png"},"statistics":[{"team":{"id":35,"name":"Bournemouth","logo":"https:\\/\\/media.api-sports.io\\/football\\/teams\\/35.png"},"league":{"id":39,"name":"Premier League","country":"England","logo":"https:\\/\\/media.api-sports.io\\/football\\/leagues\\/39.png","flag":"https:\\/\\/media.api-sports.io\\/flags\\/gb.svg","season":2015},"games":{"appearences":0,"lineups":0,"minutes":0,"number":null,"position":"Defender","rating":null,"captain":false},"substitutes":{"in":0,"out":0,"bench":0},"shots":{"total":null,"on":null},"goals":{"total":0,"conce

In [6]:
from json import loads
#Data2020 is complete, next step is to decide how to model and keep which part of the data
#data2020Json = list(map(lambda i: data2020[i].json(),data2020))
tableau=[]
for i in range(len(data)):
    tableau.append(i)
#Format data to json
jsonData=list(map(lambda i: loads(data[i]),tableau))
len(jsonData)

38

In [7]:
len(jsonData[37]['response'])

12

In [8]:
#Create prototype array which will be column names in the df
proto= ["id"\
, "firstname"\
, "lastname"\
, "age"\
, "nationality"\
, "height"\
, "weight"\
, "injured"\
, "teamID"\
, "teamname"\
, "appearences"\
, "lineups"\
, "minutes"\
, "position"\
, "rating"\
, "captain"\
, "subIn"\
, "subOut"\
, "bench"\
, "totalShots"\
, "shotsOn"\
, "totalGoals"\
, "goalsConceded"\
, "goalAssists"\
, "goalSaves"\
, "totalPasses"\
, "keyPasses"\
, "passAccuracy"\
, "totalTackles"\
, "blocks"\
, "interceptions"\
, "duelsTotal"\
, "duelsWon"\
, "dribbleAttempts"\
, "dribblesSuccess"\
, "dribblesPast"\
, "foulsDrawn"\
, "foulsCommitted"\
, "cardsYel"\
, "cardsYelRed"\
, "cardsRed"\
, "penaltyWon"\
, "penaltyCommited"\
, "penaltyScored"\
, "penaltyMissed"\
, "penaltySaved"]

In [9]:
#finalPageItems
pagesNum = len(jsonData)
finalPageItems = len(jsonData[pagesNum-1]['response'])
finalPageItems

12

In [10]:
finalArr = []
#Keys to iterate through
playerKeys = ['id', 'firstname', 'lastname', 'age', 'nationality', 'height', 'weight', 'injured']
#Create a dict to optimize code
stat = {
    'team':['id','name'],
    'games':['appearences','lineups','minutes','position','rating','captain'],
    'substitutes':['in','out','bench'],
    'shots':['total','on'],
    'goals':['total','conceded','assists','saves'],
    'passes':['total','key','accuracy'],
    'tackles':['total','blocks','interceptions'],
    'duels':['total','won'],
    'dribbles':['attempts','success','past'],
    'fouls':['drawn','committed'],
    'cards':['yellow','yellowred','red'],
    'penalty':['won','commited','scored','missed','saved']
}

#To change before every extraction
iterable = 0
pagesNum = len(jsonData)
finalPageItems = len(jsonData[pagesNum-1]['response'])
for i in range(pagesNum):
    
    #To adjust for the last page
    if i == pagesNum-1: 
        iterable = finalPageItems
    else:
        iterable = 20
        
    for j in range(iterable):
        to_fill = []
        
        for pkey in playerKeys:
            print(i,j,pkey)
            to_fill.append(jsonData[i]['response'][j]['player'][pkey])
        for key in stat:
            for innerkey in stat[key]:
                to_fill.append(jsonData[i]['response'][j]['statistics'][0][key][innerkey])        
        finalArr.append(to_fill)

0 0 id
0 0 firstname
0 0 lastname
0 0 age
0 0 nationality
0 0 height
0 0 weight
0 0 injured
0 1 id
0 1 firstname
0 1 lastname
0 1 age
0 1 nationality
0 1 height
0 1 weight
0 1 injured
0 2 id
0 2 firstname
0 2 lastname
0 2 age
0 2 nationality
0 2 height
0 2 weight
0 2 injured
0 3 id
0 3 firstname
0 3 lastname
0 3 age
0 3 nationality
0 3 height
0 3 weight
0 3 injured
0 4 id
0 4 firstname
0 4 lastname
0 4 age
0 4 nationality
0 4 height
0 4 weight
0 4 injured
0 5 id
0 5 firstname
0 5 lastname
0 5 age
0 5 nationality
0 5 height
0 5 weight
0 5 injured
0 6 id
0 6 firstname
0 6 lastname
0 6 age
0 6 nationality
0 6 height
0 6 weight
0 6 injured
0 7 id
0 7 firstname
0 7 lastname
0 7 age
0 7 nationality
0 7 height
0 7 weight
0 7 injured
0 8 id
0 8 firstname
0 8 lastname
0 8 age
0 8 nationality
0 8 height
0 8 weight
0 8 injured
0 9 id
0 9 firstname
0 9 lastname
0 9 age
0 9 nationality
0 9 height
0 9 weight
0 9 injured
0 10 id
0 10 firstname
0 10 lastname
0 10 age
0 10 nationality
0 10 height
0 10 

5 0 nationality
5 0 height
5 0 weight
5 0 injured
5 1 id
5 1 firstname
5 1 lastname
5 1 age
5 1 nationality
5 1 height
5 1 weight
5 1 injured
5 2 id
5 2 firstname
5 2 lastname
5 2 age
5 2 nationality
5 2 height
5 2 weight
5 2 injured
5 3 id
5 3 firstname
5 3 lastname
5 3 age
5 3 nationality
5 3 height
5 3 weight
5 3 injured
5 4 id
5 4 firstname
5 4 lastname
5 4 age
5 4 nationality
5 4 height
5 4 weight
5 4 injured
5 5 id
5 5 firstname
5 5 lastname
5 5 age
5 5 nationality
5 5 height
5 5 weight
5 5 injured
5 6 id
5 6 firstname
5 6 lastname
5 6 age
5 6 nationality
5 6 height
5 6 weight
5 6 injured
5 7 id
5 7 firstname
5 7 lastname
5 7 age
5 7 nationality
5 7 height
5 7 weight
5 7 injured
5 8 id
5 8 firstname
5 8 lastname
5 8 age
5 8 nationality
5 8 height
5 8 weight
5 8 injured
5 9 id
5 9 firstname
5 9 lastname
5 9 age
5 9 nationality
5 9 height
5 9 weight
5 9 injured
5 10 id
5 10 firstname
5 10 lastname
5 10 age
5 10 nationality
5 10 height
5 10 weight
5 10 injured
5 11 id
5 11 firstname

10 0 injured
10 1 id
10 1 firstname
10 1 lastname
10 1 age
10 1 nationality
10 1 height
10 1 weight
10 1 injured
10 2 id
10 2 firstname
10 2 lastname
10 2 age
10 2 nationality
10 2 height
10 2 weight
10 2 injured
10 3 id
10 3 firstname
10 3 lastname
10 3 age
10 3 nationality
10 3 height
10 3 weight
10 3 injured
10 4 id
10 4 firstname
10 4 lastname
10 4 age
10 4 nationality
10 4 height
10 4 weight
10 4 injured
10 5 id
10 5 firstname
10 5 lastname
10 5 age
10 5 nationality
10 5 height
10 5 weight
10 5 injured
10 6 id
10 6 firstname
10 6 lastname
10 6 age
10 6 nationality
10 6 height
10 6 weight
10 6 injured
10 7 id
10 7 firstname
10 7 lastname
10 7 age
10 7 nationality
10 7 height
10 7 weight
10 7 injured
10 8 id
10 8 firstname
10 8 lastname
10 8 age
10 8 nationality
10 8 height
10 8 weight
10 8 injured
10 9 id
10 9 firstname
10 9 lastname
10 9 age
10 9 nationality
10 9 height
10 9 weight
10 9 injured
10 10 id
10 10 firstname
10 10 lastname
10 10 age
10 10 nationality
10 10 height
10 10 

14 16 lastname
14 16 age
14 16 nationality
14 16 height
14 16 weight
14 16 injured
14 17 id
14 17 firstname
14 17 lastname
14 17 age
14 17 nationality
14 17 height
14 17 weight
14 17 injured
14 18 id
14 18 firstname
14 18 lastname
14 18 age
14 18 nationality
14 18 height
14 18 weight
14 18 injured
14 19 id
14 19 firstname
14 19 lastname
14 19 age
14 19 nationality
14 19 height
14 19 weight
14 19 injured
15 0 id
15 0 firstname
15 0 lastname
15 0 age
15 0 nationality
15 0 height
15 0 weight
15 0 injured
15 1 id
15 1 firstname
15 1 lastname
15 1 age
15 1 nationality
15 1 height
15 1 weight
15 1 injured
15 2 id
15 2 firstname
15 2 lastname
15 2 age
15 2 nationality
15 2 height
15 2 weight
15 2 injured
15 3 id
15 3 firstname
15 3 lastname
15 3 age
15 3 nationality
15 3 height
15 3 weight
15 3 injured
15 4 id
15 4 firstname
15 4 lastname
15 4 age
15 4 nationality
15 4 height
15 4 weight
15 4 injured
15 5 id
15 5 firstname
15 5 lastname
15 5 age
15 5 nationality
15 5 height
15 5 weight
15 5 i

18 19 id
18 19 firstname
18 19 lastname
18 19 age
18 19 nationality
18 19 height
18 19 weight
18 19 injured
19 0 id
19 0 firstname
19 0 lastname
19 0 age
19 0 nationality
19 0 height
19 0 weight
19 0 injured
19 1 id
19 1 firstname
19 1 lastname
19 1 age
19 1 nationality
19 1 height
19 1 weight
19 1 injured
19 2 id
19 2 firstname
19 2 lastname
19 2 age
19 2 nationality
19 2 height
19 2 weight
19 2 injured
19 3 id
19 3 firstname
19 3 lastname
19 3 age
19 3 nationality
19 3 height
19 3 weight
19 3 injured
19 4 id
19 4 firstname
19 4 lastname
19 4 age
19 4 nationality
19 4 height
19 4 weight
19 4 injured
19 5 id
19 5 firstname
19 5 lastname
19 5 age
19 5 nationality
19 5 height
19 5 weight
19 5 injured
19 6 id
19 6 firstname
19 6 lastname
19 6 age
19 6 nationality
19 6 height
19 6 weight
19 6 injured
19 7 id
19 7 firstname
19 7 lastname
19 7 age
19 7 nationality
19 7 height
19 7 weight
19 7 injured
19 8 id
19 8 firstname
19 8 lastname
19 8 age
19 8 nationality
19 8 height
19 8 weight
19 8 

23 8 firstname
23 8 lastname
23 8 age
23 8 nationality
23 8 height
23 8 weight
23 8 injured
23 9 id
23 9 firstname
23 9 lastname
23 9 age
23 9 nationality
23 9 height
23 9 weight
23 9 injured
23 10 id
23 10 firstname
23 10 lastname
23 10 age
23 10 nationality
23 10 height
23 10 weight
23 10 injured
23 11 id
23 11 firstname
23 11 lastname
23 11 age
23 11 nationality
23 11 height
23 11 weight
23 11 injured
23 12 id
23 12 firstname
23 12 lastname
23 12 age
23 12 nationality
23 12 height
23 12 weight
23 12 injured
23 13 id
23 13 firstname
23 13 lastname
23 13 age
23 13 nationality
23 13 height
23 13 weight
23 13 injured
23 14 id
23 14 firstname
23 14 lastname
23 14 age
23 14 nationality
23 14 height
23 14 weight
23 14 injured
23 15 id
23 15 firstname
23 15 lastname
23 15 age
23 15 nationality
23 15 height
23 15 weight
23 15 injured
23 16 id
23 16 firstname
23 16 lastname
23 16 age
23 16 nationality
23 16 height
23 16 weight
23 16 injured
23 17 id
23 17 firstname
23 17 lastname
23 17 age
23

27 8 lastname
27 8 age
27 8 nationality
27 8 height
27 8 weight
27 8 injured
27 9 id
27 9 firstname
27 9 lastname
27 9 age
27 9 nationality
27 9 height
27 9 weight
27 9 injured
27 10 id
27 10 firstname
27 10 lastname
27 10 age
27 10 nationality
27 10 height
27 10 weight
27 10 injured
27 11 id
27 11 firstname
27 11 lastname
27 11 age
27 11 nationality
27 11 height
27 11 weight
27 11 injured
27 12 id
27 12 firstname
27 12 lastname
27 12 age
27 12 nationality
27 12 height
27 12 weight
27 12 injured
27 13 id
27 13 firstname
27 13 lastname
27 13 age
27 13 nationality
27 13 height
27 13 weight
27 13 injured
27 14 id
27 14 firstname
27 14 lastname
27 14 age
27 14 nationality
27 14 height
27 14 weight
27 14 injured
27 15 id
27 15 firstname
27 15 lastname
27 15 age
27 15 nationality
27 15 height
27 15 weight
27 15 injured
27 16 id
27 16 firstname
27 16 lastname
27 16 age
27 16 nationality
27 16 height
27 16 weight
27 16 injured
27 17 id
27 17 firstname
27 17 lastname
27 17 age
27 17 nationality

31 13 firstname
31 13 lastname
31 13 age
31 13 nationality
31 13 height
31 13 weight
31 13 injured
31 14 id
31 14 firstname
31 14 lastname
31 14 age
31 14 nationality
31 14 height
31 14 weight
31 14 injured
31 15 id
31 15 firstname
31 15 lastname
31 15 age
31 15 nationality
31 15 height
31 15 weight
31 15 injured
31 16 id
31 16 firstname
31 16 lastname
31 16 age
31 16 nationality
31 16 height
31 16 weight
31 16 injured
31 17 id
31 17 firstname
31 17 lastname
31 17 age
31 17 nationality
31 17 height
31 17 weight
31 17 injured
31 18 id
31 18 firstname
31 18 lastname
31 18 age
31 18 nationality
31 18 height
31 18 weight
31 18 injured
31 19 id
31 19 firstname
31 19 lastname
31 19 age
31 19 nationality
31 19 height
31 19 weight
31 19 injured
32 0 id
32 0 firstname
32 0 lastname
32 0 age
32 0 nationality
32 0 height
32 0 weight
32 0 injured
32 1 id
32 1 firstname
32 1 lastname
32 1 age
32 1 nationality
32 1 height
32 1 weight
32 1 injured
32 2 id
32 2 firstname
32 2 lastname
32 2 age
32 2 na

36 0 age
36 0 nationality
36 0 height
36 0 weight
36 0 injured
36 1 id
36 1 firstname
36 1 lastname
36 1 age
36 1 nationality
36 1 height
36 1 weight
36 1 injured
36 2 id
36 2 firstname
36 2 lastname
36 2 age
36 2 nationality
36 2 height
36 2 weight
36 2 injured
36 3 id
36 3 firstname
36 3 lastname
36 3 age
36 3 nationality
36 3 height
36 3 weight
36 3 injured
36 4 id
36 4 firstname
36 4 lastname
36 4 age
36 4 nationality
36 4 height
36 4 weight
36 4 injured
36 5 id
36 5 firstname
36 5 lastname
36 5 age
36 5 nationality
36 5 height
36 5 weight
36 5 injured
36 6 id
36 6 firstname
36 6 lastname
36 6 age
36 6 nationality
36 6 height
36 6 weight
36 6 injured
36 7 id
36 7 firstname
36 7 lastname
36 7 age
36 7 nationality
36 7 height
36 7 weight
36 7 injured
36 8 id
36 8 firstname
36 8 lastname
36 8 age
36 8 nationality
36 8 height
36 8 weight
36 8 injured
36 9 id
36 9 firstname
36 9 lastname
36 9 age
36 9 nationality
36 9 height
36 9 weight
36 9 injured
36 10 id
36 10 firstname
36 10 lastna

In [11]:
len(finalArr)
#len(proto)

752

In [12]:
#TO NOTE: Year in query is season XXXX/XXXX+1
#Example if 2020 is past as param, the data returned is from the season 2020/2021
#Create a 2d array
arr = np.array(finalArr)
arr

array([[18015, 'Elliott', 'Ward', ..., None, None, None],
       [18774, 'Wes', 'Morgan', ..., 0, 0, None],
       [18879, 'Andrew', 'Surman', ..., 0, 0, None],
       ...,
       [82810, 'Connor', 'Dymond', ..., None, None, None],
       [82834, 'Oliver Gregory', 'Shenton', ..., None, None, None],
       [105863, 'Djair Terraii Carl', 'Parfitt-Williams', ..., None,
        None, None]], dtype=object)

In [13]:
#create the df
df = pd.DataFrame(arr, columns = proto)
df

,id,firstname,lastname,age,nationality,height,weight,injured,teamID,teamname,...,foulsDrawn,foulsCommitted,cardsYel,cardsYelRed,cardsRed,penaltyWon,penaltyCommited,penaltyScored,penaltyMissed,penaltySaved
0,18015,Elliott,Ward,36,England,185 cm,83 kg,False,35,Bournemouth,...,None,None,0,0,0,None,None,None,None,None
1,18774,Wes,Morgan,37,Jamaica,186 cm,101 kg,False,46,Leicester,...,15,27,3,0,0,None,1,0,0,None
2,18879,Andrew,Surman,35,England,178 cm,73 kg,False,35,Bournemouth,...,20,26,4,0,0,None,None,0,0,None
3,18982,Glenn,Murray,38,England,183 cm,80 kg,False,35,Bournemouth,...,18,18,2,0,0,None,None,0,1,None
4,19176,Tommy,Elphick,34,England,180 cm,85 kg,False,35,Bournemouth,...,8,10,1,0,0,None,1,0,0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
747,57460,Joshua Paul,Robson,24,England,180 cm,73 kg,False,746,Sunderland,...,None,None,0,0,0,None,None,None,None,None
748,82230,Jack Anthony,Dunn,28,England,172 cm,67 kg,False,40,Liverpool,...,None,None,0,0,0,None,None,None,None,None
749,82810,Connor,Dymond,28,England,175 cm,None,False,52,Crystal Palace,...,None,None,0,0,0,None,None,None,None,None
750,82834,Oliver Gregory,Shenton,25,England,175 cm,77 kg,False,75,Stoke City,...,None,None,0,0,0,None,None,None,None,None


In [14]:
gb = df.groupby(df['lastname']).count()#=='Kane'].goalAssists
df[df['lastname']=='Vorm']

,id,firstname,lastname,age,nationality,height,weight,injured,teamID,teamname,...,foulsDrawn,foulsCommitted,cardsYel,cardsYelRed,cardsRed,penaltyWon,penaltyCommited,penaltyScored,penaltyMissed,penaltySaved
80,160,Michel,Vorm,38,Netherlands,183 cm,84 kg,False,47,Tottenham,...,0,0,0,0,0,None,None,0,0,0


In [15]:
df['season'] = '2015/2016'

In [16]:
#Export the df to a csv
df.to_csv('data2015&2016.csv')

In [17]:
counter

38